In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, LeakyReLU
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.layers.normalization import BatchNormalization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

Using TensorFlow backend.


In [2]:
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018
#Model
NO_EPOCHS = 50
BATCH_SIZE = 64
IS_LOCAL = False

import os

if(IS_LOCAL):
    PATH="../input/fashionmnist/"
else:
    PATH="../input/"
print(os.listdir(PATH))


['fashion-24l', 'fashionmnist', 'test-dataset', 'augmented-fashionmnist', 'fashion-noise-120000']


In [3]:
train_file = '../input/fashion-24l/fashion_newdata.csv'
train_label = '../input/fashion-24l/fashion_newlabel.csv'
test_file  = '../input/fashionmnist/fashion-mnist_test.csv'

X = pd.read_csv(train_file,header=None)
y = pd.read_csv(train_label, header = None)
test_data = pd.read_csv(test_file)
print(X.shape)
print(test_data.shape)

(120269, 784)
(10000, 785)


In [4]:
# data preprocessing
def data_preprocessing(raw):
    out_y = keras.utils.to_categorical(raw.label, NUM_CLASSES)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

In [5]:
X_test, y_test = data_preprocessing(test_data)
print(X_test.shape)

(10000, 28, 28, 1)


In [6]:
train_data = np.array(train_data)
X = train_data[:,1:]
y  = train_data[:,784]
print(X.shape)
print(y.shape)


NameError: name 'train_data' is not defined

In [7]:
print(y.shape)
y = y.astype('int')
y = keras.utils.to_categorical(y, NUM_CLASSES)
print(y.shape)

(120269, 1)
(120269, 10)


In [8]:
X = X/255.0

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [10]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [11]:
print("Fashion MNIST train -  rows:",X_train.shape[0]," columns:", X_train.shape[1:4])
print("Fashion MNIST valid -  rows:",X_val.shape[0]," columns:", X_val.shape[1:4])
print("Fashion MNIST test -  rows:",X_test.shape[0]," columns:", X_test.shape[1:4])

Fashion MNIST train -  rows: 96215  columns: (784,)
Fashion MNIST valid -  rows: 24054  columns: (784,)
Fashion MNIST test -  rows: 10000  columns: (28, 28, 1)


In [12]:
x = X_train.reshape(96000, 28,28)
def plot(img):
    plt.imshow(img)
    plt.show()

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [13]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_train = X_train.reshape(96215, 28,28 ,1)
X_val = X_val.reshape(24054, 28, 28, 1)
print(X_train.shape)
print(X_val.shape)

(96215, 28, 28, 1)
(24054, 28, 28, 1)


In [14]:
# Model
'''model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
# Add dropouts to the model
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# Add dropouts to the model
model.add(Dropout(0.3))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])'''

"model = Sequential()\n# Add convolution 2D\nmodel.add(Conv2D(32, kernel_size=(3, 3),\n                 activation='relu',\n                 kernel_initializer='he_normal',\n                 input_shape=(IMG_ROWS, IMG_COLS, 1)))\nmodel.add(MaxPooling2D((2, 2)))\n# Add dropouts to the model\nmodel.add(Dropout(0.25))\nmodel.add(Conv2D(64, \n                 kernel_size=(3, 3), \n                 activation='relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n# Add dropouts to the model\nmodel.add(Dropout(0.25))\nmodel.add(Conv2D(128, (3, 3), activation='relu'))\n# Add dropouts to the model\nmodel.add(Dropout(0.4))\nmodel.add(Flatten())\nmodel.add(Dense(128, activation='relu'))\n# Add dropouts to the model\nmodel.add(Dropout(0.3))\nmodel.add(Dense(NUM_CLASSES, activation='softmax'))\n\n\nmodel.compile(loss=keras.losses.categorical_crossentropy,\n              optimizer='adam',\n              metrics=['accuracy'])"

In [15]:
# Model
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
#model.add(LeakyReLU(alpha=0.05))
model.add(BatchNormalization())

# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(LeakyReLU(alpha=0.05))
model.add(BatchNormalization())

# Add dropouts to the model
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
# Add dropouts to the model
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# Add dropouts to the model
model.add(Dropout(0.3))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 13, 13, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 5, 5, 64)          256       
__________

In [17]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=50,
                  verbose=2,
                  validation_data=(X_val, y_val))

model.save('model_fashion_16may.h5')
model.save_weights('weights_fashion_16may.h5')

Train on 96215 samples, validate on 24054 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 21s - loss: 0.9362 - acc: 0.6599 - val_loss: 0.6320 - val_acc: 0.7662
Epoch 2/50
 - 16s - loss: 0.6977 - acc: 0.7435 - val_loss: 0.5509 - val_acc: 0.7960
Epoch 3/50
 - 16s - loss: 0.6315 - acc: 0.7696 - val_loss: 0.5121 - val_acc: 0.8099
Epoch 4/50
 - 17s - loss: 0.5973 - acc: 0.7814 - val_loss: 0.4957 - val_acc: 0.8161
Epoch 5/50
 - 16s - loss: 0.5679 - acc: 0.7905 - val_loss: 0.4856 - val_acc: 0.8158
Epoch 6/50
 - 16s - loss: 0.5521 - acc: 0.7962 - val_loss: 0.4682 - val_acc: 0.8217
Epoch 7/50
 - 16s - loss: 0.5368 - acc: 0.8024 - val_loss: 0.4567 - val_acc: 0.8279
Epoch 8/50
 - 16s - loss: 0.5278 - acc: 0.8072 - val_loss: 0.4474 - val_acc: 0.8326
Epoch 9/50
 - 17s - loss: 0.5166 - acc: 0.8105 - val_loss: 0.4512 - val_acc: 0.8310
Epoch 10/50
 - 17s - loss: 0.5044 - acc: 0.8140 - val_loss: 0.4626 - val_acc: 0.8243
Epoch 11/50
 - 16s - loss: 0.4985 - acc: 0.8165 - val_loss: 0

In [18]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.20785363458991052
Test accuracy: 0.9229


In [19]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_test)
#get the indices to be plotted
y_true = test_data.iloc[:, 0]

In [20]:
p = predicted_classes[:10000]
y = y_true[:10000]
correct = np.nonzero(p==y)[0]
incorrect = np.nonzero(p!=y)[0]

In [21]:
print("Correct predicted classes:",correct.shape[0])
print("Incorrect predicted classes:",incorrect.shape[0])

Correct predicted classes: 9229
Incorrect predicted classes: 771


In [22]:
test_x = pd.read_csv('../input/test-dataset/test_data.csv')
test_y = pd.read_csv('../input/test-dataset/test_label.csv')
test_x  = np.array(test_x)
test_y = np.array(test_y)
print(test_x.shape)
print(test_y.shape)
test_x = test_x.reshape(100,28,28,1)
test_y =  keras.utils.to_categorical(test_y, NUM_CLASSES)
print(test_x.shape)
print(test_y.shape)
score1 = model.evaluate(test_x,test_y, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])
#get the predictions for the test data'''
'''predicted_classes = model.predict_classes(test_x)
#get the indices to be plotted
y_true = test_y
p = predicted_classes[:10000]
y = y_true[:10000]
correct = np.nonzero(p==y)[0]
incorrect = np.nonzero(p!=y)[0]
print("Correct predicted classes:",correct.shape[0])
print("Incorrect predicted classes:",incorrect.shape[0])'''

(100, 784)
(100, 1)
(100, 28, 28, 1)
(100, 10)
Test loss: 1.5478395234793425
Test accuracy: 0.45


'predicted_classes = model.predict_classes(test_x)\n#get the indices to be plotted\ny_true = test_y\np = predicted_classes[:10000]\ny = y_true[:10000]\ncorrect = np.nonzero(p==y)[0]\nincorrect = np.nonzero(p!=y)[0]\nprint("Correct predicted classes:",correct.shape[0])\nprint("Incorrect predicted classes:",incorrect.shape[0])'